In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tweets-about-virat-kohli/Virat_Kohli.csv


In [2]:
pip install Twython

Note: you may need to restart the kernel to use updated packages.


In [3]:
# Import Libraries

from textblob import TextBlob
import sys
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import nltk
import re
import string
import seaborn as sns


from PIL import Image
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.stem import SnowballStemmer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer

In [4]:
pd.set_option('display.max_columns', None)
tweets=pd.read_csv("/kaggle/input/tweets-about-virat-kohli/Virat_Kohli.csv")
tweets.head()

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3552: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,id,conversation_id,created_at,date,time,timezone,user_id,username,name,place,tweet,language,mentions,urls,photos,replies_count,retweets_count,likes_count,hashtags,cashtags,link,retweet,quote_url,video,thumbnail,near,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest
0,1548210906655379456,1548210906655379456,2022-07-16 07:40:19 UTC,2022-07-16,07:40:19,0,1335807310560546818,klmanjit1,Manjit Nath,NaN,Most runs for India since 2020: 2265 - Virat ...,en,"[{'screen_name': 'klrahul', 'name': 'k l rahul...",[],[],0,0,0,['klrahul'],[],https://twitter.com/klmanjit1/status/154821090...,False,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
1,1548210897662795778,1548210897662795778,2022-07-16 07:40:17 UTC,2022-07-16,07:40:17,0,1520621559458660352,sayad__ansari,Sayad Ansari,NaN,Strong Kohli 💪,in,[],[],[],0,0,0,[],[],https://twitter.com/sayad__ansari/status/15482...,False,https://twitter.com/mattyvincent44/status/1547...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
2,1548210802351517697,1548206381173334018,2022-07-16 07:39:54 UTC,2022-07-16,07:39:54,0,1036147676000075778,_klausxx,✖️,NaN,@Kohli_Spy @BCCI em ledhu,hu,[],[],[],0,0,0,[],[],https://twitter.com/_klausxx/status/1548210802...,False,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'screen_name': 'Kohli_Spy', 'name': 'S P Y',...",NaN,NaN,NaN,NaN
3,1548210792008339456,1548210792008339456,2022-07-16 07:39:51 UTC,2022-07-16,07:39:51,0,1521809171821174784,officialskyfair,SkyFair,NaN,Monty Panesar on why BCCI can't afford to 'dro...,en,[],['https://skyfair.news/indian-cricket-board-mi...,[],0,0,0,"['skyfair', 'montypanesar', 'viratkohli', 'eng...",[],https://twitter.com/officialskyfair/status/154...,False,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
4,1548210783196041224,1548210783196041224,2022-07-16 07:39:49 UTC,2022-07-16,07:39:49,0,1456176133255348231,hoshyarpakistan,hoshyarpakistan,NaN,"Babar’s message to Kohli, what did Irfan Patha...",en,[],['https://hoshyarpakistan.com/sports/babars-me...,[],0,0,0,[],[],https://twitter.com/hoshyarpakistan/status/154...,False,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN


In [5]:
tweets.drop_duplicates(inplace=True)

In [6]:
# before removing the non-english tweets
print(tweets.shape)
 
# removing all the tweets expect the
# non-english tweets
tweets = tweets[tweets['language'] == 'en']
print("After removing non-english Tweets")
 
# only the number of english tweets
print(tweets.shape) 

(20001, 36)
After removing non-english Tweets
(13218, 36)


In [7]:
def remove_rt(x): return re.sub('RT @\w+: ', " ", x)
 
def rt(x): return re.sub(
    "(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", x)
 
tweets["tweet"] = tweets.tweet.map(remove_rt).map(rt)
tweets["tweet"] = tweets.tweet.str.lower()

In [8]:
print(tweets.shape)

(13218, 36)


In [9]:
tweets[['polarity', 'subjectivity']] = tweets['tweet'].apply(
	lambda Text: pd.Series(TextBlob(Text).sentiment))

for index, row in tweets['tweet'].iteritems():
	score = SentimentIntensityAnalyzer().polarity_scores(row)
	neg = score['neg']
	neu = score['neu']
	pos = score['pos']
	comp = score['compound']
	
	if neg > pos:
		tweets.loc[index, 'sentiment'] = "negative"
	elif pos > neg:
		tweets.loc[index, 'sentiment'] = "positive"
	else:
		tweets.loc[index, 'sentiment'] = "neutral"
		
	tweets.loc[index, 'neg'] = neg
	tweets.loc[index, 'neu'] = neu
	tweets.loc[index, 'pos'] = pos
	tweets.loc[index, 'compound'] = comp

In [10]:
tweets[["tweet", "sentiment", "polarity",
		"subjectivity", "neg", "neu", "pos"]].head(5)

,tweet,sentiment,polarity,subjectivity,neg,neu,pos
0,most runs for india since 2020 2265 virat ...,neutral,0.500000,0.500000,0.000,1.000,0.000
3,monty panesar on why bcci can t afford to dro...,negative,0.000000,0.000000,0.130,0.870,0.000
4,babar s message to kohli what did irfan patha...,neutral,0.000000,0.000000,0.000,1.000,0.000
5,stay strong viratkohli indvseng viratkohl...,positive,0.433333,0.733333,0.000,0.548,0.452
7,ashish nehra backs virat kohli amidst his poor...,negative,-0.400000,0.600000,0.323,0.677,0.000


In [11]:
total_pos = len(tweets.loc[tweets['sentiment'] == "positive"])
total_neg = len(tweets.loc[tweets['sentiment'] == "negative"])
total_neu = len(tweets.loc[tweets['sentiment'] == "neutral"])
total_tweets = len(tweets)
print("Total Positive Tweets % : {:.2f}"
	.format((total_pos/total_tweets)*100))
print("Total Negative Tweets % : {:.2f}"
	.format((total_neg/total_tweets)*100))
print("Total Neutral Tweets % : {:.2f}"
	.format((total_neu/total_tweets)*100))

Total Positive Tweets % : 53.90
Total Negative Tweets % : 21.77
Total Neutral Tweets % : 24.33
